### Executing for the first time

- 1. Update git submodules 
```
git submodule update --init --recursive
```
- 2. Install necessary python packages with `pip`
```
cd monitoring-ecosystem-resilience
pip install .
```

- 3. `earthengine authenticat`

In [1]:
import sys
import os
sys.path.append("../../sbo")
sys.path.append("monitoring-ecosystem-resilience")

import sys
import matplotlib.pyplot as plt
import numpy as np

import torch

import pyro
import pyro.contrib.gp as gp
import pyro.distributions as dist
from pyro.nn import PyroSample, PyroModule
from pyro.infer import Trace_ELBO

import pickle

# Import SBO
import sbo
import utilities

# Import pyveg
from pyveg.scripts.optimize_gee_threshold import optimize_threshold

CONST_STDOUT = sys.stdout

## Helper functions

In [2]:
# Disable
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = CONST_STDOUT

## Target function

In [3]:
class ThresholdTarget(sbo.TargetFunction):
    
    def eval(self, x):
                
        results = []
        
        no_x = x.shape[0]
                
        for x_i in range(no_x):

            x_int = int(x[x_i].detach().numpy())
            
            blockPrint()
            
            result_x = optimize_threshold(x_int)
            
            enablePrint()
            
            results.append(result_x)
        
        return -torch.FloatTensor(results)

target = ThresholdTarget([[0, 765]])

## Parameters

In [4]:
num_simulations = 10

bo_steps = 10 + 1

opti_num_steps = 100
opti_params = {"lr": 0.1}
optimizer = pyro.optim.Adam(opti_params)
loss = Trace_ELBO()

return_site = "EI"

# Acquisition function optimizer
acqf_optimizer = torch.optim.Adam
acqf_opti_num_steps = 100
acqf_opti_lr = 0.5

num_candidates = 10

# Sampling
num_samples = 5

# Initial random points
N_train_points = 1

## BO simulations

In [5]:
results = None

for sim_i in range(num_simulations):
    
    try:
        X_train = torch.rand(N_train_points, 1)*765
        y_train = target.eval(X_train)

        pyro.clear_param_store()

        # GP kernel
        kernel = gp.kernels.Matern52(
            input_dim = X_train.shape[1], lengthscale = 100*torch.ones(X_train.shape[1]))

        # Semi paramteric GP model
        model = sbo.GPRegressionModule(X_train, y_train, kernel)

        # Defining GP's parameters
        model.gp.kernel.lengthscale = PyroSample(dist.Uniform(0, 100).expand([X_train.shape[1]]).to_event())
        model.gp.kernel.variance = PyroSample(dist.Uniform(0, 50))
        model.gp.noise = PyroSample(dist.Uniform(0, 1))

        guide = None
        xmins = np.zeros([bo_steps, 2], np.float32)

        for i in range(bo_steps):

            guide, predict, losses = sbo.step(model, guide, optimizer, loss, target, acqf_optimizer, 
                                              opti_num_steps=opti_num_steps, acqf_opti_num_steps=acqf_opti_num_steps,
                                              acqf_opti_lr=acqf_opti_lr, num_samples=num_samples, 
                                              num_candidates=num_candidates,
                                              return_site=return_site)

        print("Sim {} Step {} Value = {}".format(sim_i, model.X[-1], model.y[-1]))

        if sim_i == 0:
            results = np.vstack((model.X.squeeze().numpy(), model.y.numpy())).transpose()
        else:
            results = np.dstack((results, 
                                 np.vstack((model.X.squeeze().numpy(), model.y.numpy())).transpose()))
    except:
        sim_i -= 1


Sim 0 Step tensor([662.8342]) Value = -0.0
Sim 1 Step tensor([17.3417]) Value = -0.0
Sim 2 Step tensor([690.3842]) Value = -0.0
Sim 3 Step tensor([32.1111]) Value = -0.0
Sim 4 Step tensor([54.9680]) Value = -0.0
Sim 5 Step tensor([495.9662]) Value = -1.0
Sim 6 Step tensor([62.6268]) Value = -0.0
Sim 7 Step tensor([686.2757]) Value = -0.0
Sim 8 Step tensor([526.9233]) Value = -0.9979338645935059
Sim 9 Step tensor([387.8515]) Value = -0.09710744023323059


Pickling the results

In [6]:
output = open('bo_10_results.pkl', 'wb')
pickle.dump(results, output)
output.close()